# Utility for reading images

In [1]:
import tensorflow as tf
import os
import pathlib
import random
import PIL


In [2]:
path = 'C:\\Users\\jreuber\\Documents'
image_dims = [300,300]

In [3]:
def preprocess_image(image):

    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize_images(image, [192, 192])
    image /= 255.0  # normalize to [0,1] range

    return image

In [4]:
data_root = pathlib.Path(os.path.join(path, 'images'))
image_uris = list(data_root.glob('**/*.jpg'))
image_uris = [str(image) for image in image_uris]

In [5]:
print(image_uris)

['C:\\Users\\jreuber\\Documents\\images\\20180630_092333.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092335.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092337.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092339.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092341.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092505.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092930.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092940.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092943.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092954.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_092959.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_103225.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_103244.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_103322.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_103410.jpg', 'C:\\Users\\jreuber\\Documents\\images\\20180630_104203.jpg', 'C:\\Us

In [6]:
image_ds = tf.data.Dataset.from_tensor_slices(image_uris).map(tf.read_file)
print(image_ds)

with tf.Session().as_default():
    tfrec = tf.data.experimental.TFRecordWriter(os.path.join(path, 'dest_images', 'images.tfrec'))
    #tfrec = tf.data.experimental.TFRecordWriter('images.tfrec')
    tfrec.write(image_ds).run()
    
    image_ds = tf.data.TFRecordDataset(os.path.join(path, 'dest_images', 'images.tfrec')).map(preprocess_image)
    it = image_ds.make_one_shot_iterator()
    image_tensor = it.get_next()
    image = tf.Session().run(image_tensor)
    
    import IPython.display as display
    display.display(display.Image(image))


<DatasetV1Adapter shapes: (), types: tf.string>
